In [1]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests

# List of famous UAE cities
uae_cities = [
    "Dubai", "Abu Dhabi", "Sharjah", "Ajman", "Ras Al Khaimah", 
    "Fujairah", "Umm Al Quwain", "Al Ain"
]

# Define all headers
headers = [
    "1.Name", "2.Country", "4.Population (most recent available)", 
    "7. Cost of living (Weekly)", "7.1 Monthly consumption", "7.2 Family cost (month)", 
    "8.Tourist Arrivals (Annual) (number)", "9.Cashless Economy (Card payments dominant) (Yes, no, No)", 
    "17. Internet Speed (mb/s)", "22. Education Level", "23. Average Income (Monthly)", 
    "26. People Density", "30. No_of_Hospitals", "30.1 Best hospital", 
    "33. Remote Work Locations (wework, coworking, etc.)", "34. A/C (Yes in airbnb/hotels?)", 
    "37.Sim (monthly cost for 20GB data)", "38. airbnb /night", "39. coffee /cup", 
    "40. dinner (average restaurant meal)", "b_text_Best Air Carrier", 
    "b_text_Best Coffee Place", "b_text_Best Online Shop", "b_text_Best Taxi", 
    "b_text_Best Telecom", "b_text_Best University", "cc 3. Family Score Index", 
    "cc 5.Total Score", "cc 6.Quality of Life Index", "cc 18. Air Quality Index", 
    "cc 19. Safety", "cc 20. Security", "cc 21. Racial Fairness (GPT, /5)", 
    "cc 29. Traffic Safety (motor vehicle accidents)", "cc 31. Happiness Index", 
    "cc 35. Foreigner Friendly (tourism board, information services, websites, language, friendliness) (ignore)", 
    "cc Rating Internet", "cc sort_Hijab ease", "City_Image", "City_Image blur", 
    "City_Location", "City_population_text", "Continent", "d_Average Monthly Rent Description", 
    "d_Climate description", "d_Connection description", "d_CostOfLiving description", 
    "d_Cultural Diversity Description", "d_Demography description", 
    "d_Economy descrption", "d_Education description", "d_Food description", 
    "d_Health description", "d_Overview description", "d_Quality of Life Index Description", 
    "d_Safety description", "d_Tourism description", "d_Transport description", 
    "d_Unemployment Rate Description", "extra For Expath (month)", 
    "extra For Local (month)", "extra Hotel (day)", "extra Winter max/min", 
    "LL 0.1 Technology", "LL 0.2 People & health", "LL 0.3 other", 
    "LL 12.Climate (Tropical, Desert/dry, temperate, continental, polar)", 
    "LL 14. Halal food", "LL 15.Food Culture (seafood, Meat-lovers, rice dominant, plant based, protein heavy, carb heavy)", 
    "LL 24. 25. Culture list", "LL-Political Situation", "LL-Walkability Distance", 
    "Mosque Year", "resturant meal", "Schengen Visa", "uu Average Monthly Rent", 
    "uu Cultural Diversity", "uu extra N_free internet", "uu extra Summer max/min", 
    "uu Hotel (month)", "uu Summer High", "uu Unemployment Rate", "uu Winter Low", "Walkablity score", 
    "isPublished"
]

# Sort headers numerically based on the prefix number
def sort_key(header):
    try:
        # Extract the number from the header name
        return int(header.split(".")[0])
    except ValueError:
        # If no number is found, place it at the end
        return float('inf')

# Sort headers
headers_sorted = sorted(headers, key=sort_key)

# Create an empty DataFrame with the sorted headers
df = pd.DataFrame(columns=headers_sorted)

# Add UAE cities to the "1.Name" column
df["1.Name"] = uae_cities

# Function to fetch HD image link from Unsplash
def get_unsplash_image(city_name):
    access_key = "kG4FMYTAUywrQ0PsprkAgz1UtGiMBBh5APlbJwqUZ4c"  # Replace with your Unsplash API access key
    url = f"https://api.unsplash.com/search/photos?query={city_name} city&client_id={access_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["results"]:
            return data["results"][0]["urls"]["regular"]  # Return HD image link
    return None

# Add City_Image and isPublished columns
df["City_Image"] = df["1.Name"].apply(get_unsplash_image)
df["isPublished"] = "No"

# Save the DataFrame to a CSV file
df.to_csv("UAECitiesFile.csv", index=False)

print("CSV file 'UAECitiesFile.csv' created successfully with sorted headers!")

CSV file 'UAECitiesFile.csv' created successfully with sorted headers!


In [ ]:
import csv
from typing import Dict, Any
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
import dotenv
dotenv.load_dotenv()
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

# Initialize Google Search API
search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

# Function to search Google for a specific field
def search_google_about_a_field(city: str, parameter: str, question: str | None, possible_answers: list[str] | None = None) -> str:
    question = f"What is the {parameter} of {city}?" if question is None else question
    search_result = tool.run(question)
    llm = OpenAI(temperature=0.01)

    if possible_answers is None:
        prompt = PromptTemplate.from_template(
            "Given the following context, only reply with the exact {parameter} and nothing else. Do not hallucinate. Do not make up and answer irrelevant details. Answer using data type parseable values. If the answer is a number, write the answer in numeric values without text.\n<context> {search_results} </context>.\nHelpful Answer:"
        )
        chain = prompt | llm
        response = chain.invoke(
            {
                "parameter": parameter,
                "search_results": search_result
            }
        )
    else:
        prompt = PromptTemplate.from_template(
            "Given the following context, only reply with the exact value of \"{parameter}\" and nothing else. If it's a number, just give me the number. If it's a currency, convert the currency by today's rate to USD and return the number (no need to mention USD or other currencies). Do not hallucinate. Do not make up an answer. Make sure the answer is one of the ({possible_answers}). Decide for yourself which option matches the answers closely.\n<context> {search_results} </context>.\nHelpful Answer ({possible_answers}):"
        )
        chain = prompt | llm
        response = chain.invoke(
            {
                "parameter": parameter,
                "search_results": search_result,
                "possible_answers": ','.join(possible_answers)
            }
        )
    return response

# Function to search Google for descriptive fields
def search_google_about_a_field_descriptive(city: str, parameter: str, question: str | None) -> str:
    question = f"What is the {parameter} of {city}?" if question is None else question
    search_result = tool.run(question)
    llm = OpenAI(temperature=0.01)

    prompt = PromptTemplate.from_template(
        "Given the following context, provide a detailed description of {parameter} in {city}. Do not provide a single value or number. Instead, describe the context, trends, or characteristics related to {parameter}.\n<context> {search_results} </context>.\nHelpful Answer:"
    )
    chain = prompt | llm
    response = chain.invoke(
        {
            "parameter": parameter,
            "search_results": search_result
        }
    )
    return response

# Function to process a row for a city
def process_row(city_name: str) -> Dict[str, Any]:
    row_data = dict()
    row_data['1.Name'] = city_name
    row_data['2.Country'] = "1534155542320x254264304014721020"  # Fixed country value
    row_data['Continent'] = "1530600735921x438356809707683840"  # Fixed continent value

    # Numeric fields
    row_data['7. Cost of living (Weekly)'] = search_google_about_a_field(city_name, "Cost of Living (Weekly)", f"Cost of Living (Weekly) in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['7.2 Family cost (month)'] = search_google_about_a_field(city_name, "Monthly Cost of living for a family of 4", f"Monthly Cost of living for a family of 4 in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['8.Tourist Arrivals (Annual) (number)'] = search_google_about_a_field(city_name, "Annual number of tourists", f"Annual number of tourists in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['17. Internet Speed (mb/s)'] = search_google_about_a_field(city_name, "Internet Speed", None).replace("\n", "").strip().replace(",", "")
    row_data['cc 18. Air Quality Index'] = search_google_about_a_field(city_name, "Air Quality Index", f"Air Quality Index in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['23. Average Income (Monthly )'] = search_google_about_a_field(city_name, "Average monthly Income in USD", None).replace("\n", "").strip().replace(",", "")
    row_data['26. People Density'] = search_google_about_a_field(city_name, "Population Density", None).replace("\n", "").strip().replace(",", "")
    row_data['cc 29. Traffic Safety (motor vehicle accidents)'] = search_google_about_a_field(city_name, "Traffic Safety", f"Traffic Safety in {city_name}", None).replace("\n", "").strip().replace(",", "")

    row_data['30. No_of_Hospitals'] = search_google_about_a_field(city_name, "Number of Hospitals", None).replace("\n", "").strip().replace(",", "")
    row_data['30.1 Best hospital'] = search_google_about_a_field(city_name, "Best Hospital", None).replace("\n", "").strip().replace(",", "")
    row_data['37.Sim (monthly cost for 20GB data)'] = search_google_about_a_field(city_name, "Sim Card Cost", f"Sim Card Cost in {city_name} for 20GB data", None).replace("\n", "").strip().replace(",", "")
    row_data['38. airbnb /night'] = search_google_about_a_field(city_name, "Airbnb Cost per Night", f"Airbnb Cost per Night in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['39. coffee /cup'] = search_google_about_a_field(city_name, "Coffee Cost per Cup", f"Coffee Cost per Cup in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['40. dinner (average restaurant meal)'] = search_google_about_a_field(city_name, "Dinner Cost", f"Average Dinner Cost in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['cc 31. Happiness Index'] = search_google_about_a_field(city_name, "Happiness Index", f"Happiness Index in {city_name}", None).replace("\n", "").strip().replace(",", "")
    row_data['Walkablity score'] = search_google_about_a_field(city_name, "Walkability score", f"Walkability score in {city_name}", None).replace("\n", "").strip().replace(",", "")

    # Descriptive fields
    row_data['d_Climate description'] = search_google_about_a_field_descriptive(city_name, "Climate", f"Climate in {city_name}")
    row_data['d_Connection description'] = search_google_about_a_field_descriptive(city_name, "Internet Connectivity", f"Internet Connection in {city_name}")
    row_data['d_CostOfLiving description'] = search_google_about_a_field_descriptive(city_name, "Cost of Living", f"Cost of Living in {city_name}")
    row_data['d_Cultural Diversity Description'] = search_google_about_a_field_descriptive(city_name, "Cultural Diversity", f"Cultural Diversity in {city_name}")
    row_data['d_Demography description'] = search_google_about_a_field_descriptive(city_name, "Demography", f"Demography in {city_name}")
    row_data['d_Economy descrption'] = search_google_about_a_field_descriptive(city_name, "Economy", f"Economy in {city_name}")
    row_data['d_Education description'] = search_google_about_a_field_descriptive(city_name, "Education", f"Education in {city_name}")
    row_data['d_Food description'] = search_google_about_a_field_descriptive(city_name, "Food", f"Food in {city_name}")
    row_data['d_Health description'] = search_google_about_a_field_descriptive(city_name, "Health", f"Health in {city_name}")
    row_data['d_Overview description'] = search_google_about_a_field_descriptive(city_name, "Overview", f"Overview of {city_name}")
    row_data['d_Safety description'] = search_google_about_a_field_descriptive(city_name, "Safety", f"Safety in {city_name}")
    row_data['d_Tourism description'] = search_google_about_a_field_descriptive(city_name, "Tourism", f"Tourism in {city_name}")
    row_data['d_Transport description'] = search_google_about_a_field_descriptive(city_name, "Transport", f"Transport in {city_name}")

    return row_data

# Function to save data to CSV
def save_to_csv(data: Dict[str, Any], filename: str = 'UAECitiesFile_processed.csv'):
    file_exists = False
    try:
        with open(filename, 'r'):
            file_exists = True
    except FileNotFoundError:
        pass

    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

# Process all cities in the UAECitiesFile.csv
with open('UAECitiesFile.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        city_name = row['1.Name']
        print(f"Processing {city_name}...")
        city_data = process_row(city_name)
        save_to_csv(city_data, 'UAECitiesFile_processed.csv')
        print(f"Processed and saved data for {city_name}.")

print("All cities processed and saved to UAECitiesFile_processed.csv.")